## Load all anchor boxes for cars and pedestrians

In [64]:
RANDOM_SEED = 42
SAMPLE_DATA_PERCENTAGE = 1.0
MIN_IOU_REQUIRED = 0.7  # The minimum IOU required to be a good detection (ie higher is more strict)
NUM_PROPOSALS = 300
CHECKPOINT_DIR = './idd_cars_drop_checkpoints_with_nms'

%matplotlib inline

%load_ext autoreload
%autoreload 2

import os
import torch as t
from model import FasterRCNNVGG16
from trainer import FasterRCNNTrainer
import numpy as np
import pickle
import h5py
from tqdm.auto import tqdm, trange
import matplotlib.pyplot as plt

from data.util import  read_image
from utils.vis_tool import vis_bbox
from utils import array_tool as at
from utils.config import opt
from utils.idd_utils import get_annotations
from model.utils.bbox_tools import bbox_iou

from data.dataset import Dataset, TestDataset, KittiDataset
from torch.utils.data import DataLoader
from utils.vis_tool import vis_bbox, vis_image
from utils.idd_utils import IndiaDrivingDataset
from ood_metrics import auroc, plot_roc, calc_metrics
from sklearn import svm
import pickle
import itertools
import torch
from model.faster_rcnn import nograd
from data.dataset import preprocess
from utils.mahalanobis import Mahal_Dist


import warnings
warnings.filterwarnings(action='once')

np.random.seed(RANDOM_SEED)

torch.multiprocessing.set_sharing_strategy('file_system')  # Fix ancdata error

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [65]:
opt.voc_data_dir = "/home/tadenoud/Documents/kitti/VOC2012/"
kitti_train_dataset = KittiDataset(opt, split='train')
kitti_train_dataloader = DataLoader(kitti_train_dataset, batch_size=1, num_workers=4)

In [66]:
bboxes = []
labels = []
img_shapes = []

for img, img_shape, bbox, label, difficult in tqdm(kitti_train_dataloader, total=len(kitti_train_dataloader)):
    labels.append(at.tonumpy(label))
    bboxes.append(at.tonumpy(bbox))
    img_shapes.append(img_shape)

In [67]:
print(img_shapes[0])

[tensor([370]), tensor([1224])]


In [68]:
labels = np.squeeze(np.concatenate(labels, axis=1))
bboxes = np.squeeze(np.concatenate(bboxes, axis=1))

ped_mask = labels == 1
car_mask = labels == 0

In [129]:
from sklearn.cluster import KMeans

def calc_scales(boxes, num_scales=3):
    wh = [(b[3] - b[1], b[2] - b[0]) for b in boxes]
    area = np.prod(wh, axis=1)
    
    kmeans = KMeans(n_clusters=num_scales)
    kmeans.fit(np.reshape(area, (-1, 1)))
    
    clusters = kmeans.cluster_centers_ / 16 / 16
    return sorted(list(np.squeeze(np.round(clusters).astype(np.int))))

## Cars and Pedestrian boxes

In [130]:
print(calc_scales(bboxes, num_scales=4))

[9, 59, 148, 261]


## Cars detector only

In [131]:
print(calc_scales(bboxes[car_mask], num_scales=4))

[12, 69, 162, 267]


## Pedestrian detector only

In [132]:
print(calc_scales(bboxes[ped_mask], num_scales=4))

[7, 40, 81, 169]


In [ ]:
8, 32, 64, 128, 256